# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [7]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [8]:
browser = webdriver.Chrome(executable_path="C:/Users/Username/Desktop/program/python/1st-PyCrawlerMarathon/chromedriver")
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    time.sleep(SCROLL_PAUSE_TIME)
    last_height=browser.execute_script("return document.body.scrollHeight")
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height==last_height:
        break
print('complete')

'''
提示：可參考以下的Stack Overflow: 
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
'''

complete


'\n提示：可參考以下的Stack Overflow: \nhttps://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166\n'

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [9]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        https://zhuanlan.zhihu.com/p/30911642
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [10]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


已協調口罩製造商自即日起開三班趕工，初估口罩每日產能可達420萬片，政府相關部門將統一徵購、統一調控、統一售價，最終零售價格將由中央流行疫情指揮中心發佈，口罩零售價格可望調降。
中央流行疫情指揮中心今天宣布國內出現第9例武漢肺炎確診病例，衛福部長陳時中表示，新增第9例確診個案，27日有症狀、28日到醫院，今天確診為陽性；經濟部表示，今天為開工第一天，口罩廠商已恢復生產，並協調廠商24小時三班制生產。
官員指出，政府已宣布全面徵購國內廠商的每日製造產量，為了避免供需失衡、哄抬價格，年後口罩產線開工後，相關部門將統一徵購、統一調控、統一售價，經濟部會召集業者統一價格，後續價格會再調降。
經濟部表示，武漢肺炎持續，口罩銷量超過平常十倍，自1月29日到30日，政府會陸續供應470萬片口罩，提供給在醫療院所內，或是周圍的康是美、屈臣氏、維康、杏一、躍獅、健康人生等通路，同樣是零售散賣。
其他人也看了這些...
相關新聞...
----------------------------------------------------------------------

[71] 國民ETF0050明除息　每股配2.9元、殖利率達3.1%

▲受武漢肺炎疫情影響，鼠年股市一片綠油油中，僅醫療生技股逆勢上揚。（圖／記者湯興漢攝）
記者周康玉／台北報導
儘管台股今日寫下最慘開紅盤日的歷史紀錄，有國民ETF之稱的元大台灣50（0050）跌幅達5.68%或5.5元，相對抗震。0050明（31）日將除息，每股配發2.9元，以今日收盤價92.15元計算，殖利率達3.1%，收益分配發放日為3月6日。
0050分別於1月7日、1月20日進行第一、第二階段公告，每股配發2.9元，除息日訂為1月31日、收益分配發放日為3月6日。以2019年兩次評價配息各為0.7與2.9元，累計配發3.6元，配息率達3.82%，同時期（2019）含息報酬率為28.09%。
台灣首檔ETF「元大台灣50 ETF」（0050）成立於2003年6月25日，不僅是現今規模最大，且受益人數突破10萬人的ETF。截至1月20日，規模達到新台幣633億元，歷年平均配息2.29元（2005年開始配息以來），平均填息天數僅46天(不含本次配息)，尤其2019年7月19日除息日一開盤便填息達陣，創下2003年成立以來最快填息紀錄
對於有